In [2]:
%matplotlib inline
import os
import glob
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [3]:
url = 'https://worksheets.codalab.org/rest/bundles/0x9d8c9e97326746459b4eb6564324d444/contents/blob/summarization_round2_sum09_spliced.csv'
df = pd.read_csv(url, header=0, index_col=None)
print(df.shape)
df.head(n=3)


(160, 179)


,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,Expiration,NumberOfSimilarHITs,LifetimeInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.IDX0,Input.CTX0,Input.TYPE0,Input.OUT0,Input.LOGP0,Input.IDX1,Input.CTX1,Input.TYPE1,Input.OUT1,Input.LOGP1,Input.IDX2,Input.CTX2,Input.TYPE2,Input.OUT2,Input.LOGP2,Input.IDX3,Input.CTX3,Input.TYPE3,Input.OUT3,Input.LOGP3,Input.IDX4,Input.CTX4,Input.TYPE4,Input.OUT4,Input.LOGP4,Input.IDX5,Input.CTX5,Input.TYPE5,Input.OUT5,Input.LOGP5,Input.IDX6,Input.CTX6,Input.TYPE6,Input.OUT6,Input.LOGP6,Input.IDX7,Input.CTX7,Input.TYPE7,Input.OUT7,Input.LOGP7,Input.IDX8,Input.CTX8,Input.TYPE8,Input.OUT8,Input.LOGP8,Input.IDX9,Input.CTX9,Input.TYPE9,Input.OUT9,Input.LOGP9,Input.IDX10,Input.CTX10,Input.TYPE10,Input.OUT10,Input.LOGP10,Input.IDX11,Input.CTX11,Input.TYPE11,Input.OUT11,Input.LOGP11,Input.IDX12,Input.CTX12,Input.TYPE12,Input.OUT12,Input.LOGP12,Input.IDX13,Input.CTX13,Input.TYPE13,Input.OUT13,Input.LOGP13,Input.IDX14,Input.CTX14,Input.TYPE14,Input.OUT14,Input.LOGP14,Input.IDX15,Input.CTX15,Input.TYPE15,Input.OUT15,Input.LOGP15,Input.IDX16,Input.CTX16,Input.TYPE16,Input.OUT16,Input.LOGP16,Input.IDX17,Input.CTX17,Input.TYPE17,Input.OUT17,Input.LOGP17,Input.IDX18,Input.CTX18,Input.TYPE18,Input.OUT18,Input.LOGP18,Input.IDX19,Input.CTX19,Input.TYPE19,Input.OUT19,Input.LOGP19,Input.IDX20,Input.CTX20,Input.TYPE20,Input.OUT20,Input.LOGP20,Input.IDX21,Input.CTX21,Input.TYPE21,Input.OUT21,Input.LOGP21,Input.IDX22,Input.CTX22,Input.TYPE22,Input.OUT22,Input.LOGP22,Input.IDX23,Input.CTX23,Input.TYPE23,Input.OUT23,Input.LOGP23,Input.IDX24,Input.CTX24,Input.TYPE24,Input.OUT24,Input.LOGP24,Answer.0,Answer.1,Answer.10,Answer.11,Answer.12,Answer.13,Answer.14,Answer.15,Answer.16,Answer.17,Answer.18,Answer.19,Answer.2,Answer.20,Answer.21,Answer.22,Answer.23,Answer.24,Answer.3,Answer.4,Answer.5,Answer.6,Answer.7,Answer.8,Answer.9,Approve,Reject
0,3OREP8RUT2536WD93M14W161CKNGBG,3MRAM659HIM9YKMGKOTFI483Y2CO5R,Estimate Sentence Frequency for Summarization,"Given a news article, estimate how often you w...","survey, NLP, probability, news",$1.00,Thu Nov 08 14:13:25 PST 2018,20,BatchId:3429019;OriginalHitTemplateId:920937332;,3600,259200,Thu Nov 15 14:13:25 PST 2018,NaN,NaN,336KAV9KYRMCQUW642Z0QS5NW46Y29,A1ET2J1PIP0RGO,Submitted,Thu Nov 08 14:47:51 PST 2018,Thu Nov 08 15:35:23 PST 2018,Sun Nov 11 15:35:23 PST 2018,NaN,NaN,NaN,2852,100% (16/16),100% (2/2),0% (0/0),0,five-time world champion michelle kwan withdre...,HUMAN_SUM,injury leaves kwan 's olympic hopes in limbo,-37.344,1,us business leaders lashed out wednesday at le...,HUMAN_SUM,us business attacks tough immigration law,-19.4787,2,general motors corp. said wednesday its us sal...,HUMAN_SUM,gm december sales fall ## percent,-8.8106,3,several thousand people gathered on wednesday ...,HUMAN_SUM,thousands of croatians celebrate before world ...,-20.7567,4,us first lady laura bush and us secretary of s...,HUMAN_SUM,laura bush UNKNOWN rice to attend sirleaf 's i...,-15.9281,5,"jack abramoff , a former lobbyist at the cente...",HUMAN_SUM,top republican lobbyist pleads guilty to flori...,-23.6853,6,somalia 's feuding president and parliament sp...,HUMAN_SUM,somalia rivals in compromise on seat of govern...,-26.1561,7,a ##-hour strike by airport workers in portuga...,HUMAN_SUM,portuguese airport workers strike could ground...,-18.5877,8,jose mourinho renewed his partnership with por...,HUMAN_SUM,maniche renews partnership with mourinho,-15.3636,9,hollywood is planning a new sequel to adventur...,HUMAN_SUM,hollywood shores up support for ocean 's thirteen,-36.9512,10,canada advised its nationals wednesday to avoi...,HUMAN_SUM,canada recommends avoiding travel to nepal,-17.8865,11,"us auto sales will l

In [4]:
responses = ["Invalid", "Rare", "Specific", "Average", "Typical", "Very Typical"]
response_dict = dict(zip(responses, range(len(responses))))

In [5]:
worker_counter = Counter(df['WorkerId'].values)
print(len(worker_counter))
worker_dict = dict(zip(sorted(worker_counter), range(len(worker_counter))))

96


In [6]:
ctx_counter = Counter()
for i in range(25):
    ctx_counter.update(df['Input.CTX' + str(i)].values)
print(len(ctx_counter))
ctx_dict = dict(zip(sorted(ctx_counter), range(len(ctx_counter))))

99


In [7]:
model_counter = Counter()
for i in range(25):
    model_counter.update(df['Input.TYPE' + str(i)].values)

print(model_counter)
model_dict = dict(zip(sorted(model_counter), range(len(model_counter))))

Counter({'HUMAN_SUM': 2000, 'SUM09': 2000})


In [8]:
combo_counter = defaultdict(Counter)
for i in df.index:
    for j in range(25):
        combo_counter[df.loc[i, 'Input.CTX' + str(j)]].update([(df.loc[i, 'Input.TYPE' + str(j)], df.loc[i, 'Input.OUT' + str(j)])])
print(len(combo_counter))

99


In [9]:
for key, counter in combo_counter.items():
    print(key)
    for pair, count in counter.items():
        print(pair, count)
    print()

five-time world champion michelle kwan withdrew from the #### us figure skating championships on wednesday , but will petition us skating officials for the chance to compete at the #### turin olympics .
('HUMAN_SUM', "injury leaves kwan 's olympic hopes in limbo") 20
('SUM09', 'kwan withdraws from #### us figure skating championship') 20

us business leaders lashed out wednesday at legislation that would penalize companies for employing illegal immigrants .
('HUMAN_SUM', 'us business attacks tough immigration law') 20
('SUM09', 'us business leaders slam bill to penalize illegal immigrants') 20

general motors corp. said wednesday its us sales fell ##.# percent in december and four percent in #### with the biggest losses coming from passenger car sales .
('HUMAN_SUM', 'gm december sales fall ## percent') 20
('SUM09', 'gm us sales fall ##.# percent in ####') 20

several thousand people gathered on wednesday evening on the main square in zagreb for a public draw and an open air party to c

In [10]:
combo_counter = defaultdict(Counter)
rows = []
responses_by_sent_by_model = {model: defaultdict(list) for model in model_dict}

for i in df.index:
    worker = df.loc[i, 'WorkerId']
    worker_id = 'W' + str(worker_dict[worker])
    for j in range(25):
        model = df.loc[i, 'Input.TYPE' + str(j)]        
        ctx = df.loc[i, 'Input.CTX' + str(j)]
        item_id = 'I' + str(ctx_dict[ctx])
        output = df.loc[i, 'Input.OUT' + str(j)]
        response = df.loc[i, 'Answer.' + str(j)]
        response_val = float(response_dict[response])
        rows.append([worker, worker_id, model, ctx, item_id, output, response, response_val])
        responses_by_sent_by_model[model][output].append(response)

df_out = pd.DataFrame(rows, columns=['worker', 'worker_id', 'model', 'input', 'input_id', 'output', 'response', 'response_val'])
print(df_out.shape)
df_out.head()

(4000, 8)


,worker,worker_id,model,input,input_id,output,response,response_val
0,A1ET2J1PIP0RGO,W3,HUMAN_SUM,five-time world champion michelle kwan withdre...,I23,injury leaves kwan 's olympic hopes in limbo,Invalid,0.0
1,A1ET2J1PIP0RGO,W3,HUMAN_SUM,us business leaders lashed out wednesday at le...,I91,us business attacks tough immigration law,Average,3.0
2,A1ET2J1PIP0RGO,W3,HUMAN_SUM,general motors corp. said wednesday its us sal...,I28,gm december sales fall ## percent,Specific,2.0
3,A1ET2J1PIP0RGO,W3,HUMAN_SUM,several thousand people gathered on wednesday ...,I62,thousands of croatians celebrate before world ...,Rare,1.0
4,A1ET2J1PIP0RGO,W3,HUMAN_SUM,us first lady laura bush and us secretary of s...,I92,laura bush UNKNOWN rice to attend sirleaf 's i...,Specific,2.0


In [11]:
print(set(df_out['model'].values))

{'SUM09', 'HUMAN_SUM'}


In [12]:
print(set(df_out['response_val'].values))

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0}


In [14]:
outdir = os.path.join('..', 'data', 'annotated_datasets')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'huse_summarization.csv'))
